First, importing all packages which will be necessary through the process.

In [2]:
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder
import geopy
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests 
from geopy.geocoders import Nominatim

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------

In [114]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 
import numpy as np 
import matplotlib.pyplot as plt  
import pandas as pd
from sklearn import preprocessing

import json

Using the read_html pandas feature, I have read the HTML tables on the Wikipedia page and then took the first one, Neighborhood name, Denonym, Population in 2010, Population per km², sub-neighborhoods and coordinayes.

In [115]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami", header=0)
tables = tables[0]
tables.head()

,Neighborhood,Demonym,Population2010,Population/Km²,Sub-neighborhoods,Coordinates
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257


Doing the proper adjusts to the table, by dropping Rows with Coordinates or Populatio not assigned and dropping Columns which won't be important during the process.

In [116]:
tables = tables[tables['Coordinates'].notna()]
tables = tables[tables['Population/Km²'].notna()]
tables.drop(['Demonym','Sub-neighborhoods'],1, inplace=True)
tables.head()

,Neighborhood,Population2010,Population/Km²,Coordinates
0,Allapattah,54289,4401,25.815-80.224
1,Arts & Entertainment District,11033,7948,25.799-80.190
2,Brickell,31759,14541,25.758-80.193
3,Buena Vista,9058,3540,25.813-80.192
4,Coconut Grove,20076,3091,25.712-80.257


Finally, adjusting the latitude and longitude columns to be split and longitude to be negative, as well as one specific Population data which hasn't a number format.

In [117]:
tables[['Lat','Long']] = tables.Coordinates.str.split("-",expand=True)
tables.drop('Coordinates',1, inplace=True)
tables.head()

,Neighborhood,Population2010,Population/Km²,Lat,Long
0,Allapattah,54289,4401,25.815,80.224
1,Arts & Entertainment District,11033,7948,25.799,80.190
2,Brickell,31759,14541,25.758,80.193
3,Buena Vista,9058,3540,25.813,80.192
4,Coconut Grove,20076,3091,25.712,80.257


In [118]:
tables['Lat'] = pd.to_numeric(tables['Lat'])
tables['Long'] = pd.to_numeric(tables['Long'])
tables.at[7,'Population2010']= 71000
indexNames = tables[ tables['Population/Km²'] == '-' ].index
tables.drop(indexNames , inplace=True)
tables['Population2010'] = pd.to_numeric(tables['Population2010'])
tables['Population/Km²'] = pd.to_numeric(tables['Population/Km²'])
tables['Long'] = tables.Long * (-1)
tables = tables.reset_index(drop=True)
tables.head()

,Neighborhood,Population2010,Population/Km²,Lat,Long
0,Allapattah,54289,4401,25.815,-80.224
1,Arts & Entertainment District,11033,7948,25.799,-80.190
2,Brickell,31759,14541,25.758,-80.193
3,Buena Vista,9058,3540,25.813,-80.192
4,Coconut Grove,20076,3091,25.712,-80.257


In [119]:
address = 'Miami'
geolocator = Nominatim(user_agent="exploring_Miami")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Miami are 25.7742658, -80.1936589.


Now with the data well formatted, next step is to grab data from Foursquare and add it to our Dataframe, in order to be possible to analyze it with K-Means.

In [164]:
CLIENT_ID = 'EX3NXMNX5TLNEGG5KTIXLABHKWAIQZP4WBMG4CPGTSCD0DQ4' # your Foursquare ID
CLIENT_SECRET = 'BFVHVLQ352ZNOHCBJUMSCZ2SNGZOV34TQX5TWQ2D1SHVX5NT' # your Foursquare Secret
VERSION = '20200415' # Foursquare API version
LIMIT = 3000 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [165]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [166]:
miamivenues = getNearbyVenues(names=tables['Neighborhood'],
                                   latitudes=tables['Lat'],
                                   longitudes=tables['Long']
                                  )



Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Overtown
Park West
The Roads
Upper Eastside
West Flagler
Wynwood


In [167]:
print(miamivenues.shape)
miamivenues

(435, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allapattah,25.815,-80.224,Three Fingers Liquor & Lounge,25.815408,-80.224361,Lounge
1,Allapattah,25.815,-80.224,Ross,25.815820,-80.221753,Department Store
2,Allapattah,25.815,-80.224,Showtime Boxing Gym,25.812364,-80.224504,Boxing Gym
3,Allapattah,25.815,-80.224,Conde Art Gallery,25.818671,-80.224548,Art Gallery
4,Arts & Entertainment District,25.799,-80.190,Bunnie Cakes,25.799544,-80.190953,Cupcake Shop
5,Arts & Entertainment District,25.799,-80.190,Bunbury Miami,25.798284,-80.191118,Wine Shop
6,Arts & Entertainment District,25.799,-80.190,Yodi's Threading Spa,25.800490,-80.189093,Spa
7,Arts & Entertainment District,25.799,-80.190,Plant Food + Wine Miami,25.800452,-80.192805,Restaurant
8,Arts & Entertainment District,25.799,-80.190,TRIO - Mind Body Spirit Studio,25.799573,-80.190972,Yoga Studio
9,Arts & Entertainment District,25.799,-80.190,The Daily Creative Food Co. - Miami,25.795696,-80.188683,Sandwich Place


In [124]:
miamivenues.groupby('Neighborhood').count()['Venue Category']

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allapattah,4,4,4,4,4,4
Arts & Entertainment District,20,20,20,20,20,20
Brickell,57,57,57,57,57,57
Buena Vista,25,25,25,25,25,25
Coconut Grove,3,3,3,3,3,3
Coral Way,10,10,10,10,10,10
Design District,25,25,25,25,25,25
Downtown,49,49,49,49,49,49
Edgewater,51,51,51,51,51,51


In [125]:
miamitop = pd.get_dummies(miamivenues[['Venue Category']], prefix="", prefix_sep="")
miamitop['ZNeighborhood'] = miamivenues['Neighborhood'] 
rightcolumns = [miamitop.columns[-1]] + list(miamitop.columns[:-1])
miamitop = miamitop[rightcolumns]
miamitop = miamitop.rename(columns={'ZNeighborhood':'Correct Neighborhood'})

miamitop.head()

,Correct Neighborhood,Accessories Store,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Empanada Restaurant,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,Hotel,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Print Shop,Pub,Public Art,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio
0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Allapattah,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [173]:
miamigroup = miamitop.groupby('Correct Neighborhood').mean().reset_index()

miamigrouped = miamivenues.groupby('Neighborhood').count()
miamigroup['Number of Venues'] = miamigrouped['Venue Category']
miamigrouped = miamimerge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


Neighborhood
Allapattah                        4
Arts & Entertainment District    20
Brickell                         57
Buena Vista                      25
Coconut Grove                     3
Coral Way                        10
Design District                  25
Downtown                         49
Edgewater                        51
Flagami                           6
Grapeland Heights                 7
Liberty City                      2
Little Haiti                     23
Little Havana                     9
Lummus Park                      11
Overtown                          3
Park West                        27
The Roads                         7
Upper Eastside                    7
West Flagler                      8
Wynwood                          81
Name: Venue Category, dtype: int64

In [127]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [128]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = miamigroup['Correct Neighborhood']

for ind in np.arange(miamigroup.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(miamigroup.iloc[ind, :], num_top_venues)


miamigroupcluster = miamigroup.drop('Correct Neighborhood', 1)
miamigroupcluster.head()

,Accessories Store,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Empanada Restaurant,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,Hotel,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Print Shop,Pub,Public Art,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio
0,0.0,0.000000,0.0,0.00,0.000000,0.25,0.00,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.25,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.250000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.00
1,0.0,0.000000,0.0,0.00,0.000000,0.00,0.00,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.000000,0.0,0.0,0.05,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.050000,0.000000,0.0,0.0,0.000000,0.05,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.05,0.0,0.00,0.000000,0.000000,0.0,0.05,0.050000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.050000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.150000,0.000000,0.050000,0.050000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.05,0.000000,0.0,0.000000,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.000000,0.00,0.05,0.0,0.0,0.05
2,0.0,0.035088,0.0,0.00,0.035088,0.00,0.00,0.00,0.017544,0.0,0.0,0.035088,0.035088,0.00,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.017544,0.0,0.035088,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.017544,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.017544,0.0,0.0,0.087719,0.00,0.0,0.0,0.070175,0.052632,0.00,0.035088,0.017544,0.0,0.00,0.017544,0.017544,0.0,0.017544,0.0,0.017544,0.0,0.0,0.00,0.0,0.00,0.017544,0.01754

Adding population and density data into the new table, both normalized.

In [135]:
x = tables[['Population2010']].values.astype(float)
y = tables[['Population/Km²']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
Pop_scaled = min_max_scaler.fit_transform(x)
Dens_scaled = min_max_scaler.fit_transform(y)
miamigroupcluster['Pop_scaled']=Pop_scaled
miamigroupcluster['Dens_scaled']=Dens_scaled
miamigroupcluster.head()



,Accessories Store,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Empanada Restaurant,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,Hotel,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Print Shop,Pub,Public Art,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Pop_scaled,Dens_scaled
0,0.0,0.000000,0.0,0.00,0.000000,0.25,0.00,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.25,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.250000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.00,0.700913,0.156967
1,0.0,0.000000,0.0,0.00,0.000000,0.00,0.00,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.000000,0.0,0.0,0.05,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.050000,0.000000,0.0,0.0,0.000000,0.05,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.05,0.0,0.00,0.000000,0.000000,0.0,0.05,0.050000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.050000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.150000,0.000000,0.050000,0.050000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.05,0.000000,0.0,0.000000,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.000000,0.00,0.05,0.0,0.0,0.05,0.109467,0.451862
2,0.0,0.035088,0.0,0.00,0.035088,0.00,0.00,0.00,0.017544,0.0,0.0,0.035088,0.035088,0.00,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.017544,0.0,0.035088,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.017544,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.017544,0.0,0.0,0.087719,0.00,0.0,0.0,0.070175,0.052632,0.00,0.035088,0.017544,0.0,0.00,0.017544,0.017544,0.0,0

Since we have only 21 neighborhoods, the data will be divided among 5 clusters.

In [136]:
# set number of clusters
kclusters = 5



# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(miamigroupcluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 4, 2, 2, 2, 2, 3, 2, 0], dtype=int32)

In [137]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

miamimerge = tables

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
miamimerge = miamimerge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



In [138]:
miamimerge= miamimerge.dropna()
miamimerge['Cluster Labels'] = miamimerge['Cluster Labels'].apply(np.int64)
miamimerge

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allapattah,54289,4401,25.815,-80.224,0,Department Store,Art Gallery,Lounge,Boxing Gym,Yoga Studio,Donut Shop,Fish Market,Fast Food Restaurant,Event Space,Empanada Restaurant
1,Arts & Entertainment District,11033,7948,25.799,-80.190,2,Restaurant,Gym,Yoga Studio,Spa,Coffee Shop,Cupcake Shop,Food Truck,Gym / Fitness Center,Moving Target,Paper / Office Supplies Store
2,Brickell,31759,14541,25.758,-80.193,4,Hotel,Italian Restaurant,Japanese Restaurant,Bank,Juice Bar,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Bar,Café
3,Buena Vista,9058,3540,25.813,-80.192,2,Café,Italian Restaurant,Clothing Store,Boutique,Jewelry Store,Mediterranean Restaurant,Fast Food Restaurant,Leather Goods Store,Coffee Shop,Shopping Mall
4,Coconut Grove,20076,3091,25.712,-80.257,2,Boat or Ferry,American Restaurant,Park,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Food
5,Coral Way,35062,4496,25.750,-80.283,2,Liquor Store,Mobile Phone Shop,Golf Course,Pharmacy,Seafood Restaurant,Burger Joint,Café,Dive Bar,Historic Site,Diner
6,Design District,3573,3623,25.813,-80.193,2,Café,Italian Restaurant,Clothing Store,Boutique,Jewelry Store,Mediterranean Restaurant,Fast Food Restaurant,Leather Goods Store,Coffee Shop,Shopping Mall
7,Downtown,71000,10613,25.774,-80.193,3,Hotel,Peruvian Restaurant,Italian Restaurant,Café,Cosmetics Shop,Coffee Shop,Brazilian Restaurant,Pharmacy,Lounge,Pet Service
8,Edgewater,15005,6675,25.802,-80.190,2,Restaurant,Sandwich Place,Cuban Restaurant,Gym / Fitness Center,Gym,Pizza Place,Food Truck,Coffee Shop,Breakfast Spot,Art Gallery
9,Flagami,50834,5665,25.762,-80.316,0,Seafood Restaurant,Fast Food Restaurant,Spanish Restaurant,Department Store,Bakery,Yoga Studio,Dog Run,Event Space,Empanada Restaurant,Donut Shop


Now, getting Mandolin's location and calculating its distance from all the neighborhoods centerpoints. Then, this data will be added to the dataframe.

In [159]:
mandolin = '4312 NE 2nd Ave, Miami, FL 33137'
geolocator = Nominatim(user_agent="mandolin")
locationm = geolocator.geocode(mandolin)
latitudem = locationm.latitude
longitudem = locationm.longitude
print('The geograpical coordinate of Mandolin are {}, {}.'.format(latitudem, longitudem))

The geograpical coordinate of Mandolin are 25.81577787755102, -80.19156669387755.


In [160]:
def dist (names, lat, lon):
    from math import sin, cos, sqrt, atan2, radians

    # approximate radius of earth in km
    R = 6373.0
    dist_list=[]
    for name, lat, lon in zip(names, lat, lon):
        print(name)
        lat1 = radians(latitudem)
        lon1 = radians(longitudem)
        lat2 = radians(lat)
        lon2 = radians(lon)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        dist_list.append([(
            name, 
            lat, 
            lon,
            distance)])
    distancemiami = pd.DataFrame([item for dist_list in dist_list for item in dist_list])
    distancemiami.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Distance']
    return (distancemiami)

In [161]:
miamidistance = dist(names=tables['Neighborhood'],
                                   lat=tables['Lat'],
                                   lon=tables['Long']
                                  )

Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Overtown
Park West
The Roads
Upper Eastside
West Flagler
Wynwood


In [162]:
miamidistance
miamimerge['Distance'] = miamidistance['Distance']
miamimerge

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Distance
0,Allapattah,54289,4401,25.815,-80.224,0,Department Store,Art Gallery,Lounge,Boxing Gym,Yoga Studio,Donut Shop,Fish Market,Fast Food Restaurant,Event Space,Empanada Restaurant,3.248677
1,Arts & Entertainment District,11033,7948,25.799,-80.190,2,Restaurant,Gym,Yoga Studio,Spa,Coffee Shop,Cupcake Shop,Food Truck,Gym / Fitness Center,Moving Target,Paper / Office Supplies Store,1.872783
2,Brickell,31759,14541,25.758,-80.193,4,Hotel,Italian Restaurant,Japanese Restaurant,Bank,Juice Bar,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Bar,Café,6.428227
3,Buena Vista,9058,3540,25.813,-80.192,2,Café,Italian Restaurant,Clothing Store,Boutique,Jewelry Store,Mediterranean Restaurant,Fast Food Restaurant,Leather Goods Store,Coffee Shop,Shopping Mall,0.312014
4,Coconut Grove,20076,3091,25.712,-80.257,2,Boat or Ferry,American Restaurant,Park,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Food,13.274360
5,Coral Way,35062,4496,25.750,-80.283,2,Liquor Store,Mobile Phone Shop,Golf Course,Pharmacy,Seafood Restaurant,Burger Joint,Café,Dive Bar,Historic Site,Diner,11.721493
6,Design District,3573,3623,25.813,-80.193,2,Café,Italian Restaurant,Clothing Store,Boutique,Jewelry Store,Mediterranean Restaurant,Fast Food Restaurant,Leather Goods Store,Coffee Shop,Shopping Mall,0.340687
7,Downtown,71000,10613,25.774,-80.193,3,Hotel,Peruvian Restaurant,Italian Restaurant,Café,Cosmetics Shop,Coffee Shop,Brazilian Restaurant,Pharmacy,Lounge,Pet Service,4.649163
8,Edgewater,15005,6675,25.802,-80.190,2,Restaurant,Sandwich Place,Cuban Restaurant,Gym / Fitness Center,Gym,Pizza Place,Food Truck,Coffee Shop,Breakfast Spot,Art Gallery,1.540520
9,Flagami,50834,5665,25.762,-80.316,0,Seafood Restaurant,Fast Food Restaurant,Spanish Restaurant,Department Store,Bakery,Yoga Studio,Dog Run,Event Space,Empanada Restaurant,Donut Shop,13.823434


In [163]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(miamimerge['Lat'], miamimerge['Long'], miamimerge['Neighborhood'], miamimerge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
folium.Marker(location=[latitudem,longitudem], icon=folium.Icon(color='lightgray', icon='home', prefix='fa')).add_to(map_clusters)
map_clusters